In [1]:
#Loaded Libraries

library(tidyverse)
library(repr)
library(tidymodels)
library(caret)
install.packages("expss")
library(expss)
library(ggplot2)
library(GGally)
library("gridExtra")
library(rvest)
library(stringr)
library(DBI)
library(dbplyr)
library(GGally)
options(repr.matrix.max.rows = 10)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

In [2]:
# Data Loaded 

cleveland_data <- read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data", 
                           col_names = c("age", 
                                         "sex", 
                                         "cp", 
                                         "trestbps", 
                                         "chol", 
                                         "fbs", 
                                         "restercg", 
                                         "thalach", 
                                         "exang", 
                                         "oldpeak", 
                                         "slope", 
                                         "ca", 
                                         "thal", 
                                         "num"))
cleveland_data$num <- as.factor(cleveland_data$num)
head(cleveland_data)

Parsed with column specification:
cols(
  age = col_double(),
  sex = col_double(),
  cp = col_double(),
  trestbps = col_double(),
  chol = col_double(),
  fbs = col_double(),
  restercg = col_double(),
  thalach = col_double(),
  exang = col_double(),
  oldpeak = col_double(),
  slope = col_double(),
  ca = col_character(),
  thal = col_character(),
  num = col_double()
)



age,sex,cp,trestbps,chol,fbs,restercg,thalach,exang,oldpeak,slope,ca,thal,num
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<fct>
63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0
56,1,2,120,236,0,0,178,0,0.8,1,0.0,3.0,0


In [5]:
# see if there is other number rather than 0-4 in the num column
cleveland_data  %>%
    select(num)  %>% 
    unlist() %>% 
    levels()

# transform num from 0-4 to 0 or 1 (1, 2, 3, 4) and store into diagnosis variable
cleveland_data <- cleveland_data %>%
                  mutate(diagnosis = dplyr::recode(num, '0' = 'absent', .default = 'present'))

head(cleveland_data)

# to see how the original data distribute
cleveland_data %>% 
    group_by(diagnosis) %>%
    summarize(n = n())

# We select all the numerical columns and diagnosis column
cleveland_data2 <- cleveland_data%>%
            select(age, trestbps, chol, thalach, oldpeak, diagnosis,num)
head(cleveland_data2)

heart_tidy <- cleveland_data2

# Above codes are from the proposal, with the tidy dataset cleveland_data2 renamed as heart_tidy

[1] "0" "1" "2" "3" "4"

age,sex,cp,trestbps,chol,fbs,restercg,thalach,exang,oldpeak,slope,ca,thal,num,diagnosis
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<fct>,<fct>
63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0,absent
67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2,present
67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1,present
37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0,absent
41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0,absent
56,1,2,120,236,0,0,178,0,0.8,1,0.0,3.0,0,absent


`summarise()` ungrouping output (override with `.groups` argument)



diagnosis,n
<fct>,<int>
absent,164
present,139


age,trestbps,chol,thalach,oldpeak,diagnosis,num
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>
63,145,233,150,2.3,absent,0
67,160,286,108,1.5,present,2
67,120,229,129,2.6,present,1
37,130,250,187,3.5,absent,0
41,130,204,172,1.4,absent,0
56,120,236,178,0.8,absent,0


# Training and Testing Datasets

Before we begin exploratory data analysis, we must divide our data into training and testing, with 75 percent of the data in the training set and 25% in the testing set. This split allows a large enough training set to build an accurate model while leaving enough data in the test set to assess the model's performance accurately. We will use our training data in the following steps, going with the testing data to be used later to determine the validity and accuracy of our model.

In [6]:
# We will use KNN Classification with 10 folds cross-validation for choosing the appropriate k value. The resulting accuracy will be calculated

# Splitting the data to training and testing data

heart_split <- initial_split(heart_tidy, prop = 0.75, strata = diagnosis)
heart_train <- training(heart_split)
heart_test <- testing(heart_split)
heart_train


age,trestbps,chol,thalach,oldpeak,diagnosis,num
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>
63,145,233,150,2.3,absent,0
67,160,286,108,1.5,present,2
41,130,204,172,1.4,absent,0
56,120,236,178,0.8,absent,0
57,120,354,163,0.6,absent,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮
45,110,264,132,1.2,present,1
68,144,193,141,3.4,present,2
57,130,131,115,1.2,present,3


# Tuning the Classifier

In order to build our classification model, we need to use the tidy model's package's k nearest neighbors capacity and modify it to get the best value for the parameter k. (the number of neighbors considered in the class vote).

We start by assigning the tune() function to the parameter value k in the knn classification model specification we'll use for tuning. Then weset up a range of 1 -23 to test wich is the best value for k in this range, 

We got a graph to campare the effects of different varaibles of the prediction. We used that graph to find the 3 varaibles to get the highest accuracy.

In a workflow, we connect our model specification with the scaling formula that was previously defined. Then, the tune grid function fits the model to each value in our parameter range.

In [69]:
# Perform 10 folds Cross-Validation for heart_train dataset to choose appropriate k
set.seed(2022)
heart_10fold <- vfold_cv(heart_train, v=10, strata = diagnosis)

In [70]:
set.seed(2022)
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbor = tune()) %>%
            set_engine("kknn") %>%
            set_mode("classification")

In [71]:
knn_recipe <- recipe(diagnosis ~ chol + thalach + oldpeak, data = heart_train) %>% # Important !!!!!!!!!!!! Using only 3 predictors (chol,thalach, oldpeak) will gain the max accuracy
            step_center(all_predictors()) %>%
            step_scale(all_predictors())

In [72]:
heart_workflow <- workflow() %>%
                add_model(knn_spec) %>%
                add_recipe(knn_recipe)

In [73]:
gridvals <- tibble(neighbors = seq(from = 1, to = 23))
heart_result <- heart_workflow %>%
                tune_grid(resamples = heart_10fold, grid = gridvals) %>%
                collect_metrics() 

# Chosing the model

In [62]:
set.seed(1)

cleveland_data3 <- cleveland_data %>%
            filter(!is.na(chol))%>%
            select(age, trestbps, chol, thalach, oldpeak, num)

heart_split <- initial_split(cleveland_data3, prop = 0.75, strata = num)
heart_train <- training(heart_split)
heart_test <- testing(heart_split)

# Create list of all predictors that we want to iterate through
lm_predictors <- c(
    'age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'num'
)

# setup results dataframe. Will have rows for each predictor and resulting RMSPE
results <- tibble()
results <- mutate(results, predictor = "", best_k = 0, rmspe = 0)

for (pred in predictors) {

    print(pred)
    

    train_data <- heart_train %>%
        select(num, all_of(pred))
    
    test_data <- heart_test %>%
        select(num, all_of(pred))
    
    heart_recipe <- recipe(num ~ ., data = train_data) %>%
       step_scale(all_predictors()) %>%
       step_center(all_predictors())
    
    heart_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>%
       set_engine("kknn") %>%
       set_mode("regression")
    
    heart_vfold <- vfold_cv(train_data, v = 5, strata = num)
    
    heart_workflow <- workflow() %>%
       add_recipe(heart_recipe) %>%
       add_model(heart_spec)
    
    gridvals <- tibble(neighbors = seq(1,40))
    
    heart_results <- heart_workflow %>%
       tune_grid(resamples = heart_vfold, grid = gridvals) %>%
       collect_metrics() %>%
       filter(.metric == "rmse") %>%
       filter(mean == min(mean))
    
    kmin <- pull(heart_results, neighbors)
    
    heart_spec_kmin <- nearest_neighbor(weight_func = "rectangular", neighbors = kmin) %>%
       set_engine("kknn") %>%
       set_mode("regression")
    
    heart_fit <- workflow() %>%
       add_recipe(heart_recipe) %>%
       add_model(heart_spec_kmin) %>%
       fit(data = train_data)
    
    rmspe_val <- heart_fit %>%
       predict(test_data) %>%
       bind_cols(test_data) %>%
       metrics(truth = num, estimate = .pred) %>%
       filter(.metric == "rmse") %>%
       select(.estimate) %>%
       pull()
    
    # Append predictor, best k, and rmspe value to the results dataframe
    results <- results %>%
        add_row(predictor = pred, best_k=kmin, rmspe = rmspe_val)
}


ERROR: Error in for (pred in predictors) {: invalid for() loop sequence


In [61]:
results %>% arrange(rmspe)

predictor,best_k,rmspe
<chr>,<dbl>,<dbl>


In [54]:
set.seed(1)


# Create list of all predictors that we want to iterate through
lm_predictors <- c(
    'age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'num'
)

# setup results dataframe. Will have rows for each predictor and resulting RMSPE
lm_results <- tibble()
lm_results <- mutate(lm_results, lm_predictor = "", lm_rmspe = 0)

for (lm_pred in lm_predictors) {

    print(lm_pred)
    

    lm_train_data <- heart_train %>%
        select(num, all_of(lm_pred))
    
    lm_test_data <- heart_test %>%
        select(num, all_of(lm_pred))
    
    heart_lm_recipe <- recipe(num ~ ., data = lm_train_data) %>%
       step_scale(all_predictors()) %>%
       step_center(all_predictors())
    
    heart_lm_spec <- linear_reg() %>%
        set_engine("lm") %>%
        set_mode("regression")

    heart_lm_fit <- workflow() %>%
       add_recipe(heart_lm_recipe) %>%
       add_model(heart_lm_spec) %>%
       fit(data = lm_train_data)
    
    lm_rmspe_val <- heart_lm_fit %>%
         predict(lm_test_data) %>%
         bind_cols(lm_test_data) %>%
         metrics(truth = num, estimate =.pred) %>%
         filter(.metric == "rmse") %>%
         select(.estimate) %>%
         pull()
    
    # Append predictor and rmspe value to the results dataframe
    lm_results <- lm_results %>%
        add_row(lm_predictor = lm_pred, lm_rmspe = lm_rmspe_val)
}

[1] "age"


Warning message in model.response(mf, "numeric"):
“using type = "numeric" with a factor response will be ignored”
Warning message in Ops.factor(y, z$residuals):
“‘-’ not meaningful for factors”


ERROR: Error: `estimate` should be a factor


In [45]:
lm_results %>%
    arrange(lm_rmspe)

lm_predictor,lm_rmspe
<chr>,<dbl>


In [52]:
set.seed(1)
# list of formulas that will be used to generate recipes iteratively
formulas <- c(
"num ~ age + trestbps",
"num ~ age + trestbps + chol",
"num ~ age + chol",
"num ~ age + chol + thalach",
"num ~ age + thalach",
"num ~ age + thalach + oldpeak ",
"num ~ age +oldpeak ",
"num ~ trestbps + chol",
"num ~ trestbps + chol + thalach",
"num ~ trestbps + thalach",
"num ~ trestbps + thalach + oldpeak ",
"num ~ trestbps + oldpeak ",
"num ~ chol + thalach",
"num ~ chol + thalach + oldpeak ",
"num ~ chol + oldpeak ",
"num ~ thalach + oldpeak "
)


# setup results dataframe. Will have rows for each predictor and resulting RMSPE
multi_lm_results <- tibble()
multi_lm_results <- mutate(multi_lm_results, lm_predictor = "", lm_rmspe = 0)

for (f in formulas) {

    print(f)
    
    # Converts string formula to formula object
    heart_spec_lm <- linear_reg() %>%
        set_engine("lm") %>%
        set_mode("regression")
    
    heart_recipe_lm <- recipe(as.formula(f), data = heart_train)
    
    heart_fit_lm <- workflow() %>%
        add_recipe(heart_recipe_lm) %>%
        add_model(heart_spec_lm) %>%
        fit(data = heart_train)
    
    lm_rmspe_val <- heart_fit_lm %>%
        predict(heart_test) %>%
        bind_cols(heart_test) %>%
        metrics(truth = num, estimate = .pred) %>%
        filter(.metric == "rmse") %>%
        select(.estimate) %>%
        pull()

    # append formula, and rmspe to results table
    multi_lm_results <- multi_lm_results %>%
        add_row(lm_predictor = f, lm_rmspe = lm_rmspe_val)

}

[1] "num ~ age + trestbps"


Warning message in model.response(mf, "numeric"):
“using type = "numeric" with a factor response will be ignored”
Warning message in Ops.factor(y, z$residuals):
“‘-’ not meaningful for factors”


ERROR: Error: `estimate` should be a factor


In [ ]:
accuracy <-  heart_result %>%
            filter(.metric == "accuracy") 

In [ ]:
accuracy_versus_k <- ggplot(accuracy, aes(x = neighbors, y = mean))+
       geom_point() +
       geom_line() +
       labs(x = "Neighbors", y = "Accuracy Estimate") +
       scale_x_continuous(breaks = seq(0, 23, by = 1)) +  # adjusting the x-axis
       scale_y_continuous(limits = c(0.4, 1.0)) +# adjusting the y-axis 
       theme(text=element_text(size=20))

accuracy_versus_k

appropriate_k <- accuracy %>%
                arrange(-mean) %>%
                slice(1) %>%
                select(neighbors) %>%
                pull()
appropriate_k

In [ ]:
set.seed(2022)
knn_spec_new <- nearest_neighbor(weight_func = "rectangular", neighbor = appropriate_k) %>%
            set_engine("kknn") %>%
            set_mode("classification")

heart_fit <- workflow() %>%
                    add_model(knn_spec_new) %>%
                    add_recipe(knn_recipe) %>%
                    fit(data = heart_train)

heart_predictions <- predict(heart_fit, heart_test) %>%
                    bind_cols(heart_test)

heart_metrics <- heart_predictions %>% 
    metrics(truth = diagnosis, estimate = .pred_class)

heart_conf_mat <- heart_predictions %>%
    conf_mat(truth =diagnosis, estimate = .pred_class)

heart_metrics
heart_conf_mat

In [8]:

# create an empty tibble to store the results
names <- colnames(heart_train)
names

[1] "age"       "trestbps"  "chol"      "thalach"   "oldpeak"   "diagnosis"
[7] "num"

In [12]:
## Textbook version
heart_subset <- heart_train %>% 
  select(age, 
         trestbps, 
         chol, 
         thalach, 
         oldpeak, 
         diagnosis)

names <- colnames(heart_subset %>% select(-diagnosis))

accuracies <- tibble(size = integer(),
                    model_string = character(),
                    accuracy = numeric())

# create a model specification
knn_spec <- nearest_neighbor(weight_func = "rectangular", 
                             neighbors = tune()) %>%
     set_engine("kknn") %>%
     set_mode("classification")

# create a 5-fold cross-validation object
heart_vfold <- vfold_cv(heart_train, v = 5, strata = diagnosis)

# store the total number of predictors
n_total <- length(names)

# stores selected predictors
selected <- c()

# for every size from 1 to the total number of predictors
for (i in 1:n_total) {
    # for every predictor still not added yet
    accs <- list()
    models <- list()
    for (j in 1:length(names)) {
        # create a model string for this combination of predictors
        preds_new <- c(selected, names[[j]])
        model_string <- paste("diagnosis", "~", paste(preds_new, collapse="+"))

        # create a recipe from the model string
        heart_recipe <- recipe(as.formula(model_string), 
                                data = heart_train) %>%
                          step_scale(all_predictors()) %>%
                          step_center(all_predictors())

        # tune the KNN classifier with these predictors, 
        # and collect the accuracy for the best K
        acc <- workflow() %>%
          add_recipe(heart_recipe) %>%
          add_model(knn_spec) %>%
          tune_grid(resamples = heart_vfold, grid = 10) %>%
          collect_metrics() %>%
          filter(.metric == "accuracy") %>%
          summarize(mx = max(mean))
        acc <- acc$mx %>% unlist()

        # add this result to the dataframe
        accs[[j]] <- acc
        models[[j]] <- model_string
    }
    jstar <- which.max(unlist(accs))
    accuracies <- accuracies %>% 
      add_row(size = i, 
              model_string = models[[jstar]], 
              accuracy = accs[[jstar]])
    selected <- c(selected, names[[jstar]])
    names <- names[-jstar]
}

accuracies

size,model_string,accuracy
<int>,<chr>,<dbl>
1,diagnosis ~ thalach,0.7323671
2,diagnosis ~ thalach+oldpeak,0.7718841
3,diagnosis ~ thalach+oldpeak+trestbps,0.7762319
4,diagnosis ~ thalach+oldpeak+trestbps+chol,0.7542029
5,diagnosis ~ thalach+oldpeak+trestbps+chol+age,0.7194203
